## Analysis 2020 Notebook 03 - Sand Compaction

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

sns.set_context('talk')
sns.color_palette('bright')
#sns.set(style="ticks", palette="bright")

%matplotlib inline

In [49]:
# IMPORT DATA:
data_path = '../../../data/compaction/'
plot_path = '../../../miscellaneous/Webinar Nov 2020/'
data_file = 'Compaction_Data.csv'


df = pd.read_csv(data_path+data_file)
#df

In [58]:
# ADD A SURVEY ID FIELD BUILT OFF THE YEAR AND MONTH PARTS OF THE EXISTING sample_dat COLUMN:

year = df['sample_dat'].str[0:4]
month = df['sample_dat'].str[6:7]

# format: month-Year
df['Survey'] = df['sample_dat'].str[5:7]+'-'+df['sample_dat'].str[0:4]

In [53]:
# DROP A FEW COLUMNS THAT ARE EITHER NOT NEEDED OR TOO SPARSELY POPULATED TO BE USEFUL:
df.drop(['OBJECTID','sheet_code','sample_tim','mbc_at_1in','mbc_at_6in','mbc_at_12i'], axis=1, inplace=True)

In [54]:
# CONVERT sample_dat FIELD TO PYTHON DATETIME:

df['Sample Date'] =  pd.to_datetime(df['sample_dat'], format='%Y/%m/%d %H:%M:%S')

In [61]:
df['Sample Date'].unique()

array(['07-2014', '09-2014', '04-2015', '08-2015', '10-2015', '02-2016',
       '05-2016', '08-2016', '10-2016', '02-2017', '04-2017', '08-2017',
       '01-2020', '04-2020', '07-2020', '10-2020'], dtype=object)

In [62]:
df['Sample Date'].unique()

array(['2014-07-14T00:00:00.000000000', '2014-07-15T00:00:00.000000000',
       '2014-09-29T00:00:00.000000000', '2015-04-28T00:00:00.000000000',
       '2015-08-04T00:00:00.000000000', '2015-10-20T00:00:00.000000000',
       '2016-02-17T00:00:00.000000000', '2016-05-10T00:00:00.000000000',
       '2016-08-09T00:00:00.000000000', '2016-10-13T00:00:00.000000000',
       '2017-02-19T00:00:00.000000000', '2017-04-27T00:00:00.000000000',
       '2017-08-04T00:00:00.000000000', '2020-01-08T00:00:00.000000000',
       '2020-04-27T00:00:00.000000000', '2020-04-28T00:00:00.000000000',
       '2020-07-13T00:00:00.000000000', '2020-10-15T00:00:00.000000000',
       '2020-10-14T00:00:00.000000000'], dtype='datetime64[ns]')

In [63]:
pd.to_datetime(df['Survey'], format='%m-%Y')

0     2014-07-01
1     2014-07-01
2     2014-07-01
3     2014-07-01
4     2014-07-01
         ...    
427   2020-10-01
428   2020-10-01
429   2020-10-01
430   2020-10-01
431   2020-10-01
Name: Survey, Length: 432, dtype: datetime64[ns]

In [64]:
# ADD A FIELD THAT CONTAINS THE SURVEY DATE AS JULIAN DAYS/DATES:

# add 0.5 to bring date to whole day (Julian dates start counting at mid-day)
# use: pd.to_datetime(df['Survey'] to convert Survey field to temorary datetime for comps.

df['DateJ'] = pd.DatetimeIndex(pd.to_datetime(df['Survey'], format='%m-%Y')).floor('d').to_julian_date()+0.5

In [66]:
# ADD FIELD CONTAINING GREGORIAN DATES (CONVERT FROM JULIAN DATES: DateJ):

df['DateG'] = pd.to_datetime(df['DateJ']-0.5, unit='D', origin='julian')
#df

,transect_I,area,sample_dat,year,season,swc_at_1in,swc_at_6in,swc_at_12i,Sample Date,Survey,DateJ,DateG
0,T01,Treatment,2014/07/14 00:00:00,2014,summer,10,40,120,2014-07-14,07-2014,2456840.0,2014-07-01
1,T02,Treatment,2014/07/14 00:00:00,2014,summer,20,50,120,2014-07-14,07-2014,2456840.0,2014-07-01
2,T03,Treatment,2014/07/14 00:00:00,2014,summer,10,50,110,2014-07-14,07-2014,2456840.0,2014-07-01
3,T04,Treatment,2014/07/14 00:00:00,2014,summer,10,50,150,2014-07-14,07-2014,2456840.0,2014-07-01
4,T05,Treatment,2014/07/14 00:00:00,2014,summer,10,100,175,2014-07-14,07-2014,2456840.0,2014-07-01
...,...,...,...,...,...,...,...,...,...,...,...,...
427,T12,Treatment,2020/10/14 00:00:00,2020,autumn,0,200,>300,2020-10-14,10-2020,2459124.0,2020-10-01
428,T13,Treatment,2020/10/14 00:00:00,2020,autumn,50,300,>300,2020-10-14,10-2020,2459124.0,2020-10-01
429,T14,Treatment,2020/10/15 00:00:00,2020,autumn,0,>300,>300,2020-10-15,10-2020,2459124.0,2020-10-01
430,T15,Treatment,2020/10/15 00:00:00,2020,autumn,0,300,>300,2020-10-15,10-2020,2459124.0,2020-10-01


In [67]:
# SET XTICKS TO CONTINUOUS VALUES (DateJ) FOR EXPLORATORY REGRESSION PLOTS WHERE TIME CANNOT BE USED AS PREDICTOR:
# NEW TICK LABELS USE Survey FIELD AND MAP TO DATEJ POSITIONS ALONG X-AXIS

xtics = df.DateJ.unique()
xticlbls = df['Survey'].unique()